# Pair Automation using ```pipeline2```

## Imports and plot settings

Run this once

In [ ]:
import os
import json

import numpy as np
from matplotlib import pyplot as plt
import h5py as h5
import specpy as sp

from pipeline2 import PipelineLevels, AcquisitionPipeline, DefaultNameHandler, HDF5RichData, HDF5DataStore
from pipeline2.taskgeneration import (AcquisitionTaskGenerator, JSONFileConfigLoader, DefaultFOVSettingsGenerator,
                                      AlreadyImagedFOVFilter,
                                      DefaultScanOffsetsSettingsGenerator, DefaultLocationKeeper,
                                      DefaultLocationRemover, DefaultStageOffsetsSettingsGenerator,
                                      SpiralOffsetGenerator, NewestSettingsSelector, NewestDataSelector,
                                      BoundingBoxLocationGrouper, DefaultScanModeSettingsGenerator,
                                      DefaultScanFieldSettingsGenerator)
from pipeline2.detection import (LegacySpotPairFinder, SimpleSingleChannelSpotDetector, SimpleNucleusMidplaneDetector,
                                 SimpleLegacyFocusHold, SimpleLocationRepeater)
from pipeline2.imspector import ImspectorConnection, get_current_stage_coords
from pipeline2.stoppingcriteria import TimedStoppingCriterion
from pipeline2.taskgeneration import StitchedNewestDataSelector


# plotting params
%matplotlib inline
plt.rcParams['figure.figsize'] = [7, 7]

## Code definition for a single acquisition

Run this once

In [ ]:
def run_pipeline(save_path, im, ov_json, det_jsons, start_coords, ov_moves, hours_to_image=12, sample_name=None, ignore_z_overlap_check=True):
    
    # make output dir 
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    
    # init levels and name handler
    pll = PipelineLevels('overview', 'detail')
    name_handler = DefaultNameHandler(save_path, pll)

    # abuse name_handler to get h5 filename
    # calling with empty index gives just the random hash
    h5file_name = name_handler.get_path([]).replace('.msr', '.h5')

    # open resulting h5 file in context manager
    with h5.File(h5file_name, mode='a') as h5fd:
        data_store = HDF5DataStore(h5fd, pll)

        # init pipeline
        pl = (AcquisitionPipeline('nucleus-midplane-pipeline' if sample_name is None else sample_name)
                .withImspectorConnection(ImspectorConnection(im).withVerbose())
                .withPipelineLevels(pll)
                .withNameHandler(name_handler)
                .withAddedStoppingCondition(TimedStoppingCriterion(hours_to_image * 60 * 60))
             )

        # attach h5 data store
        pl.withDataStorage(data_store)

        # overview task generator: same settings every time
        atg_overview = (AcquisitionTaskGenerator(pll.overview, 
                        DefaultLocationRemover(
                            JSONFileConfigLoader([ov_json])
                        ),
                        DefaultStageOffsetsSettingsGenerator(SpiralOffsetGenerator().withFOV(ov_moves).withStart(start_coords)),
                        DefaultStageOffsetsSettingsGenerator(SimpleLegacyFocusHold(NewestDataSelector(pl, pll.overview)).withVerbose(True))
                    )
                    .withDelay(.4))

        # init detector in overview
        detector = (SimpleNucleusMidplaneDetector(StitchedNewestDataSelector(pl, pll.overview), n_classes=2)
                    .withVerbose()
                    .withPlot()
                    .withFilter({'area' : (100, 10000)})
                    .withFOVExpansion(1.4)
                   )

        # detail task generation: settings from last overview + new FOV + grouped detections
        # optionally repeat measurement to check reproducibility

        atg_detail = AcquisitionTaskGenerator(pll.detail,
                                 DefaultLocationKeeper(NewestSettingsSelector(pl, pll.overview)),
                                 DefaultLocationRemover(
                                     JSONFileConfigLoader(det_jsons, None, False)
                                 ),
                                 DefaultScanFieldSettingsGenerator(detector))


        atg_detail.withDelay(.2)

        # add filter to ignore new bounding boxes that have intersection-over-union >0.5
        # with any already scanned bounding boxes
        flt = AlreadyImagedFOVFilter(pl, pll.detail, 0.5, ignore_z_overlap_check)
        atg_detail.withFilters(flt)

        # overview callback: re-add overview
        pl.withCallbackAtLevel(atg_overview, pll.overview)
        # overview callback 2: detect & do detail
        pl.withCallbackAtLevel(atg_detail, pll.overview)

        # call overview atg once to add first measurement to pipeline
        atg_overview(pl)

        # GO
        pl.run()

## 1) initialize acquisition list and connect to Imspector

Run this to initialize the acquisition queue and connect to imspector
Re-run to clear queued acquisitions

In [ ]:
# init acq. list
acquisitions = []

# imspector object
im = sp.Imspector()

## 2) Enqueue an acquisition

Set the parameters for a new acquisition and add it to the queue.
** Acquisition will be run at the current stage coordinates**

In [ ]:
params = {}

### CHANGE ME!
params['save_path'] = 'D:/AUTOMATION/auto-sir/2020705_IMR90_young_untreated/2020-07-08_rep3/'


##### (OPTIONALLY) CHANGE US!

# how long to image
params['hours_to_image'] = 24

### PARAMETER FILES
# paths of the parameters files
# we can use multiple for the SETD measurement (e.g. to do both a 2d and 3d STED acq.)
params['ov_json'] = 'C:/Users/RESOLFT/Desktop/config_json/20200312_sir_overview_640-25.json'
params['det_jsons'] = [
    'C:/Users/RESOLFT/Desktop/config_json/20200312_sir_detail_640-35_775-30.json',
    #'C:/Users/RESOLFT/Desktop/config_json/20181002_edu_conf+sted3d_detail.json'
]

### SPIRAL MOVE SIZE ###
# Set this smaller than the overview FOV if you want to use Stitching
params['ov_moves'] = [4e-5, 4e-5] # how much to move in overview spiral

# whether to ignore the z coordinate in checking for already imaged regions
params['ignore_z_overlap_check'] = True

### Things that are set automatically
# ensure we use slashes and not backslashes
params['save_path'] = params['save_path'].replace(os.sep, '/')
# sample name, will be set automatically
params['sample_name'] = params['save_path'].strip('/').rsplit('/')[-1]
# start at current coordinates, do not change!
params['start_coords'] = get_current_stage_coords(im)

# add to queue
acquisitions.append(params)

# print the currently queued acquisitions
print("""
Queued Acquisitions:
====================
""")
for ac in acquisitions:
    print(json.dumps(ac, indent=1))

## 3) Run the acquisitions

Execute cell below to run the enqueued acquisitions

In [ ]:
# go through queued acquisitions (in reverse)
# and run them
for ac in reversed(acquisitions):
    # pass Imspector as well (we excluded it until now because otherwise, the JSON print above would not work)
    ac_t = ac
    ac_t['im'] = im
    run_pipeline(**ac)

# Reset queued acquisitions
# NB: if you cancelled a run, this might not be executed,
#     make sure to clear old acquisitions manually (step 1)
acquisitions = []